In [1]:
import gymnasium as gym
import panda_gym
from sb3_contrib import TQC
from huggingface_sb3 import load_from_hub
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
import time
import numpy as np
from panda_gym.utils import distance

In [2]:
chk = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="tqc-PandaPickAndPlace-v3.zip")
stats = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="vec_normalize.pkl")
   
env = gym.make('PandaPickAndPlace-v3', render_mode="human", autoreset=False)

sim = env.sim
robot = env.robot
task = env.task

env = DummyVecEnv([lambda: env])
env = VecNormalize.load(stats, env)
env.training = False
env.norm_reward = False

pybullet build time: Nov 28 2023 23:48:36
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot` for environment variables or `env.get_wrapper_attr('robot')` that will search the reminding wrappers.
  logger.warn(
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.task to get variables from other wrappers is deprecated and will be removed in v1.0, to get this va

In [4]:
model = TQC.load(chk,env) 
obs = env.reset()

q1 = np.array([1.1 / 4 - 0.3, 0.7 / 4 , 0.4])
q2 = np.array([1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4])
q3 = np.array([(1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4])
q4 = np.array([(1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4])

while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, dones, info = env.step(action)


    # gather data
    ee = robot.get_ee_position()
    fw = robot.get_fingers_width()
    block = task.get_achieved_goal()
    goal = np.array(sim.get_base_position("target"))
    
    state = {
        "effector_pos": ee,
        "finger_width": fw,
        "d_q1": distance(ee,q1),
        "d_q2": distance(ee,q2),
        "d_q3": distance(ee,q3),
        "d_q4": distance(ee,q4),
        "distance_to_block": distance(ee,block),
        "block_to_goal": distance(block, goal),
        "is_grasping": fw > 0.02 and fw < 0.04,
        "is_closed": fw < 0.01
    }

    done = dones[0]
    time.sleep(1)


/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


True
False
False
False
False
False
True
True
True
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
True
False
False
False
False
True
True
True
True
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
False


KeyboardInterrupt: 

In [ ]:
# robot.get_ee_position() gets us the position of the robots "fingers"
# robot.get_fingers_width() gets us the width between the robots "fingers"

# the table has dimensions (1.1, 0.7, 0.4) w/ x_offset=-0.3
# can partition the table into 4 quadrants
# q1 = (1.1 / 4 - 0.3, 0.7 / 4 , 0.4)
# q2 = (1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4)
# q3 = ((1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4)
# q4 = ((1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4)

# distance_to_q{} = d(robot.get_ee_position(), q{})

# grasping might be distance from effectors to block & a check to see if they arent completely open or closed
# distance_to_block = d(robot.get_ee_position(), desired_goal)


In [ ]:
# task.get_achieved_goal() where the block is now
# desired_goal = np.array(sim.get_base_position("target"))
# d = distance(achieved_goal, desired_goal)

In [ ]:
# looks like overall we keep the env in the wrapped form and just save the variables
# so we can mess with them elsewhere